# Prediciendo la enfermedad cardiaca usando regresión logística

El dataset Framingham Heart Study es un conjunto de datos famoso en el campo de la epidemiología y la medicina. La ciudad de Framingham, Massachusetts, en los Estados Unidos, ha sido la sede de un estudio a largo plazo sobre la salud del corazón que comenzó en 1948. Desde entonces, los datos recopilados en Framingham han sido vitales para nuestra comprensión de la enfermedad cardiovascular.

El estudio de Framingham fue uno de los primeros en identificar la hipertensión, el colesterol alto, el tabaquismo, la obesidad, la diabetes y otros factores como riesgos clave para el desarrollo de enfermedades del corazón. Los investigadores han seguido a miles de participantes durante décadas, recopilando datos sobre su salud, sus hábitos de vida y sus condiciones médicas.

## Descripción de las variables

* male: si es hombre o no (cateegorica)
* education: nivel educativo (categorica)
* currentSmoker: si fuma o no (categorica)
* cigsPerDay: numero de cigarros que fuma por dia (continua)
* BPMeds: si el paciente ha tomado medicamentos para la presion arterial (categorica)
* prevalentStroke: si el paciente ha tenido un ACV (categorica)
* prevalentHyp: si el paciente es hipertenso (cat)
* diabetes: idem diabetes
* totChol: nivel de colesterol (cont)
* sysBP: presion arterial sistolica (cont)
* diaBP: presion arterial diastolica (cont)
* BMI: Indice de masa corporal (cont)
* heartRate: frecuencia cardiaca (cont)
* glucose: nivel de glucosa (cont)
* TenYearCHD: Riesgo de 10 años de cardiopatía coronaria CHD (esta sera nuestra variable target)

Es importante recordar que aunque los datos del estudio de Framingham han proporcionado muchas ideas valiosas sobre la enfermedad del corazón, los participantes en el estudio son predominantemente de raza blanca y de un área geográfica específica, por lo que los hallazgos pueden no ser generalizables a todas las poblaciones.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('framingham.csv')
df.head()

## Analisis exploratorio y pre-procesamiento de datos

Acá se deben ver como se distirbuyen las variables, descartar las que estan correlacionadas entre si, y tratar los valores faltantes

Por simplicidad y en vista de querer evitar posibles colinealidades, vamos a descartar algunas variables que parecen ser redundantes con otras. Por ejemplo, 0 cigarros por día es equivalente a que no fume. La presió diastólica por lo general está muy correlacionada con la sistólica, y las presión alta está determinada principalmente por esta última. La variable de glucosa se eliminará también, favorenciendo a la variable categórica de la prevalencia de diabetes. 

In [ ]:
df.drop(['currentSmoker','diaBP','glucose'], axis = 1, inplace = True)

In [ ]:
df.info()

Vamos a tratar los valores faltantes eliminándolos

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

## Preparación de los datos para el modelo



Ahora, debemos tratar los atributos que son categóricos. Recordar que se debe borrar una categoría base para evitar la multicolinealidad perfecta o "trampa de la variable dummy". Esta vez lo haremos usando el argumento drop_first

In [ ]:
dummies_male = pd.get_dummies(df.male, drop_first=True) 
dummies_education = pd.get_dummies(df.education, drop_first=True)
dummies_BPMeds = pd.get_dummies(df.BPMeds, drop_first=True)
dummies_prevalentStroke= pd.get_dummies(df.prevalentStroke, drop_first=True)
dummies_prevalentHyp = pd.get_dummies(df.prevalentHyp, drop_first=True)
dummies_diabetes = pd.get_dummies(df.diabetes , drop_first=True)

Para evitar confusiones a la hora de interpretar el output del modelo, cambiamos los nombres de la columna dummie

In [ ]:
dummies_male.columns = ['male']
dummies_education.columns = ['educ_2','educ_3','educ_4']
dummies_BPMeds.columns = ['prev_BPMeds']
dummies_prevalentStroke.columns = ['prev_stroke']
dummies_prevalentHyp.columns = ['prev_hyp']
dummies_diabetes.columns = ['prev_diabetes']

Ahora, empezamos a construir las matrices de atributos y de respuesta del modelo

In [ ]:
X = df.drop(['male','education','BPMeds','prevalentStroke','prevalentHyp','diabetes','TenYearCHD'],axis=1)
X = pd.concat([X, dummies_male, dummies_education,dummies_BPMeds, dummies_prevalentStroke,dummies_prevalentHyp, dummies_diabetes], axis =1)
X

In [ ]:
y = df['TenYearCHD']
y

Ahora separamos entre entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Y ahora normalizamos la data (advertenecia: si este paso no se hace, saldrá una advertencia adelante: que el modelo no converge y se necesita aumentar el numero de iteraciones o bien, normalizar los datos, haga la prueba!)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

## Ajuste y predicción

El clásico fit y predict 

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

Imprimimos los coeficientes.

In [ ]:
print(f'Coeficientes : {model.coef_}')
print(f'Sesgo : {model.intercept_}')

In [ ]:
X.columns

In [ ]:
pd.DataFrame(model.coef_, columns =X.columns )

Obtenemos los odd ratios a partir de la exponencial de los coeficientes. Recordar que los valores mayores 1 influye positivamente en la probabilidad del target (enfermedad coronaria)

In [ ]:
import numpy as np
pd.DataFrame(np.exp(model.coef_), columns =X.columns )

¿Cómo anda el sobreajuste?

In [ ]:
print(f'Test accuracy: {model.score(X_test,y_test)}')
print(f'Train accuracy: {model.score(X_train,y_train)}')

## Evaluación

Calcular metricas de evaluacion, ROC y AUC

In [ ]:
from sklearn.metrics import confusion_matrix

matriz_confusion = pd.DataFrame(confusion_matrix(y_test, y_pred), columns = ['Predicted Positive', 'Predicted Negative'], 
                  index=['Actual Positive', 'Actual Negative'])
matriz_confusion

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test,y_pred))
print("Precision:", precision_score(y_test, y_pred, ))
print("Recall:", recall_score(y_test,y_pred))
print("F1 Score:", f1_score(y_test,y_pred))

In [ ]:
# Area Under Curve - AUC
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
model_roc_auc = roc_auc_score(y_test, model.predict(X_test))

fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % model_roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim(([0.0, 1.0]))
plt.ylim(([0.0, 1.05]))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend()
plt.show()

¿Se puede enmendar el modelo?

¿qué puede haber pasado?

Como dice la metodología CRISP-DM, debemos ir hacia atrás

Veamos cómo anda la clase target, que fue un paso que no hicimos antes

In [ ]:
import seaborn as sns
sns.countplot(df.TenYearCHD)

Estamos frente a **datos desbalanceados**, y esto se podría haber pensando desde la comprensión del problema. La enfermedad coronaria es algo raro, no masivo, afortundamente, por lo tanto, es de esperar que se tengan datos desbalanceados en cualquier muestra. Si esto pasa, es porque el modelo está reconociendo más las clase "0" y muy poco la clase "1". Esto ya lo podemos sospechar desde los valores de la precisión y la sensibilidad.

Existen diversas técnica de balanceo de clases. Una de ellas es SMOTE, que lo que hace -coloquialmente hablando- es simular más datos de la clase desfavorecida cosa de balancear el problema.

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

clf = LogisticRegression()
model_res = clf.fit(X_train_res, y_train_res)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
model_roc_auc = roc_auc_score(y_test, model_res.predict(X_test))

fpr, tpr, thresholds = roc_curve(y_test, model_res.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='AUC (area = %0.2f)' % model_roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim(([0.0, 1.0]))
plt.ylim(([0.0, 1.05]))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend()
plt.show()

**Ejercicios**

- Calcular matriz de confusión, y medidas de evaluación del nuevo modelo balanceado
- Ver si con otra selección de variables este modelo mejora su evaluación


In [ ]:
# re-interpretando

pd.DataFrame(np.exp(model_res.coef_), columns =X.columns )